In [1]:
!pip install pandas
!pip install numpy
!pip install matplotlib

  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
  Using cached numpy-1.26.2-cp310-cp310-win_amd64.whl (15.8 MB)
  Using cached pytz-2023.3.post1-py2.py3-none-any.whl (502 kB)


You should consider upgrading via the 'C:\Users\ignac\Documents\Ignacio\Delft\TUDelft\TUDelftFiles\Y2\Q2\DataMining\recommender-systems\venv\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\ignac\Documents\Ignacio\Delft\TUDelft\TUDelftFiles\Y2\Q2\DataMining\recommender-systems\venv\Scripts\python.exe -m pip install --upgrade pip' command.


  Using cached matplotlib-3.8.2-cp310-cp310-win_amd64.whl (7.6 MB)
  Using cached Pillow-10.1.0-cp310-cp310-win_amd64.whl (2.6 MB)
  Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
  Using cached kiwisolver-1.4.5-cp310-cp310-win_amd64.whl (56 kB)
  Using cached pyparsing-3.1.1-py3-none-any.whl (103 kB)
  Using cached contourpy-1.2.0-cp310-cp310-win_amd64.whl (186 kB)


You should consider upgrading via the 'C:\Users\ignac\Documents\Ignacio\Delft\TUDelft\TUDelftFiles\Y2\Q2\DataMining\recommender-systems\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
train = pd.read_csv('data/lab2_train.csv', delimiter = ',', skipinitialspace=True)
test = pd.read_csv('data/lab2_test.csv', delimiter = ',', skipinitialspace=True)

In [4]:
train.head()

,user_from_id,user_to_id,is_like,is_match
0,1136,3141,False,False
1,2424,3174,False,False
2,1300,3590,False,False
3,800,2736,False,False
4,883,437,False,False


In [5]:
test.head()

,user_from_id,user_to_id,is_like,is_match
0,2644,2595,?,?
1,567,2412,?,?
2,2732,3187,?,?
3,783,854,?,?
4,1104,2723,?,?


# Non-negative Matrix Factorization

# Min-Hashing